# Sprint 4 Architecture Code

## Date Modified: April 9, 2019

## Last Modified By: Eric Magliarditi

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import time
import seaborn as sns
import math
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# We are utilizing a 500 day on surface, conjunction mission format.

## We are assuming the following:

### We are going to one landing location
### Stay within 200 km
### Prior to human landing, we will predeploy a MAV that fuels using ISRU and will be fully fueled when they arrive
### Also making a first mission assumption


## Independent Decision Points:

### 1. Rover Used
### 2. EDL Technique
### 3. Landing Location
### 4. Crew Size


In [2]:
decision_matrix = pd.read_excel("../Resources/decision_matrix_V_3.0.xlsx")
decision_matrix

,Decision,A,B,C,D,E
0,Rover,MSC,Mars Direct,DRM 1,Mobile Home,Commuter
1,EDL,Retropropulsion,Supersonic Decelerators,Hypersonic Aeroassist,Deployable Entry System,NaN
2,Landing Location,Jezero Crater,Midway,Northeast Syrtis,Columbia Hills,NaN
3,Crew Size,2,3,4,5,6


## Decision Matrix Processing 

In [3]:
rovers = np.array(decision_matrix.loc[0])[1:]
edl_options = np.array(decision_matrix.loc[1])[1:5]
landing_locations = np.array(decision_matrix.loc[2])[1:5]
number_of_crew_options = np.array(decision_matrix.loc[3])[1:]


## This function will create an architecture. Each architecture will contain the information as described below

In [4]:
def create_arch(rover, edl, landing_location,num_crew):
    arch ={
        'Rover': rover,
        'EDL': edl,
        'Landing Location': landing_location,
        'Number of Crew': num_crew
    }
    return arch

# Lets Begin the Full Factorial Architecture Generation Process!

In [5]:
architectures = []

for rover in rovers:
    for edl in edl_options:
        for landing_location in landing_locations:
            for num_crew in number_of_crew_options:
                arch = create_arch(rover, edl,landing_location,num_crew)
                architectures.append(arch)


print("Total number of architectures: {}".format(len(architectures)))


Total number of architectures: 400


# Now that we have all the important enumerations, lets put it into a datafame and begin our operations

In [6]:
##create a dataframe fo the archs
df = pd.DataFrame(architectures)
df.head(10)

,EDL,Landing Location,Number of Crew,Rover
0,Retropropulsion,Jezero Crater,2,MSC
1,Retropropulsion,Jezero Crater,3,MSC
2,Retropropulsion,Jezero Crater,4,MSC
3,Retropropulsion,Jezero Crater,5,MSC
4,Retropropulsion,Jezero Crater,6,MSC
5,Retropropulsion,Midway,2,MSC
6,Retropropulsion,Midway,3,MSC
7,Retropropulsion,Midway,4,MSC
8,Retropropulsion,Midway,5,MSC
9,Retropropulsion,Midway,6,MSC


In [7]:
## CODE HERE
df_1 = df.copy()

In [8]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
EDL                 400 non-null object
Landing Location    400 non-null object
Number of Crew      400 non-null int64
Rover               400 non-null object
dtypes: int64(1), object(3)
memory usage: 12.6+ KB


# Lets Bring in DRM 5.0 as a reference

In [9]:
def nasa_DRM_5():
    rover = 'Commuter'
    edl = "Hypersonic Aeroassist"
    landing_location = "Jezero Crater"
    num_crew = 6
    arch = create_arch(rover, edl,landing_location,num_crew)
    return arch
    
# def quick_stay():
#     arch = create_arch(rover, edl, landing_location,num_crew)
#     return arch

# def cheap_stay():
#     arch = create_arch(rover, edl,landing_location,num_crew)
#     return arch


drm5 = nasa_DRM_5()
# quick_stay_arch = quick_stay()
# cheap_arch = cheap_stay()


reference_df = pd.DataFrame([drm5,
#                              quick_stay_arch,
#                              cheap_arch
                            ]) #make sure you make it a list! 
reference_df.rename(index={0:'NASA DRM 5.0',
#                            1: 'Quick Stay',
#                            2: "JPL Cheap Stay"
                           }, inplace=True)


In [10]:
reference_df

,EDL,Landing Location,Number of Crew,Rover
NASA DRM 5.0,Hypersonic Aeroassist,Jezero Crater,6,Commuter


In [11]:
df_2 = df_1.copy()

In [12]:
df_2.head()

,EDL,Landing Location,Number of Crew,Rover
0,Retropropulsion,Jezero Crater,2,MSC
1,Retropropulsion,Jezero Crater,3,MSC
2,Retropropulsion,Jezero Crater,4,MSC
3,Retropropulsion,Jezero Crater,5,MSC
4,Retropropulsion,Jezero Crater,6,MSC


# We now want to incorporating other ideas that are dependent on the variables involved with our decision matrix.


## Variables that are dependent on particular decisions include:

1. From Rover: Mass, crew capacity, payload capacity, max range, pressurized, cost, exploration area
2. From EDL: Payload, Cost, Mass, Reusable, Risk
3. From Landing Location: Science Score, (maybe distance to ISRU)
4. From Crew Size: Amount of Consumables Needed, Hab Size (Need Scaling Factor)


## Crew Size Dependencies: Consumables Needed, Hab Size

In [13]:
df_2['Total Crew Days'] = df_2['Number of Crew']*500 #500 since conjungtion assumption
reference_df['Total Crew Days'] = reference_df['Number of Crew']*500

consumables_per_crew_day_kg = (0.62 #Solid Food needed per crewmember day
                               + 0.83 #oxygen needer per crew member day
                               + 0.3 #Oxygen tank mass
                               + 4.16 #Water consumed per crew member day
                               + 0.83 #Water Tanks
                               + 20 #roughly average for water needs for hygenie and food prep
                               + 4.98 #ammount of human output that must be dealt with
                               + 1.75 #LiOH tank mass and gas mass
                              )*1.25 #25% margin added


df_2['Consumables Needed [kg]'] = df_2['Total Crew Days'].apply(lambda x: x*consumables_per_crew_day_kg) 
reference_df['Consumables Needed [kg]'] = reference_df['Total Crew Days'].apply(lambda x: x*consumables_per_crew_day_kg) 
#Source: Jones, H. W. (2019). Much Lower Launch Costs Make Resupply Cheaper Than Recycling for Space Life Support, (July 2017).



hab_scaling_factor = 25 #meters cubed per person
df_2['Habitation Volume [m^3]'] = df_2['Number of Crew']*hab_scaling_factor
reference_df['Habitation Volume [m^3]'] = 197.73

#Source: NASA Human Research Program. Minimum Acceptable Net Habitable Volume for Long-Duration Exploration Missions. April 2015
#Source URL: https://ston.jsc.nasa.gov/collections/trs/_techrep/TM-2015-218564.pdf



## Rover Dependencies: Mass, People Capacity, Payload Capacity, Max Range, Pressurized, Cost

In [14]:
rovers = {
    #List goes in following: Number of Rovers Required,
#   Mass of Rover, Number of People, Payload Capacity [mT], Max Range [km], Pressurized, cost [Billion]
    "MSC": [2,185,1,11.7,160,"No", 16], 
    "Mars Direct": [1,1400,4,26.9,750,"Yes",44],
    "DRM 1": [1,1290,0,19.4,500,"Yes", 36],
    "Mobile Home": [2,5562,6,31.9,300,"Yes",21],
    "Commuter": [2,8000,2,22.6,100,"Yes", 27]
}

df_2['Number of Rovers Needed'] = df_2['Rover'].apply(lambda x: rovers[x][0])
df_2['Rover Mass [kg]'] = df_2['Rover'].apply(lambda x: rovers[x][1])
df_2['Rover People Capacity'] = df_2['Rover'].apply(lambda x: rovers[x][2])
df_2['Rover Payload Capacity [mT]'] = df_2['Rover'].apply(lambda x: rovers[x][3])
df_2['Rover Range (km)'] = df_2['Rover'].apply(lambda x: rovers[x][4])
df_2['Rover Pressurized'] = df_2['Rover'].apply(lambda x: rovers[x][5])
df_2['Rover Cost [$B]'] = df_2['Rover'].apply(lambda x: rovers[x][6])

reference_df['Number of Rovers Needed'] = reference_df['Rover'].apply(lambda x: rovers[x][0])
reference_df['Rover Mass [kg]'] = reference_df['Rover'].apply(lambda x: rovers[x][1])
reference_df['Rover People Capacity'] = reference_df['Rover'].apply(lambda x: rovers[x][2])
reference_df['Rover Payload Capacity [mT]'] = reference_df['Rover'].apply(lambda x: rovers[x][3])
reference_df['Rover Range (km)'] = reference_df['Rover'].apply(lambda x: rovers[x][4])
reference_df['Rover Pressurized'] = reference_df['Rover'].apply(lambda x: rovers[x][5])
reference_df['Rover Cost [$B]'] = reference_df['Rover'].apply(lambda x: rovers[x][6])


## Landing Site Dependencies: Science Score, Distance to ISRU Water(TBR)

In [15]:
landing_dependencies = {
    'Jezero Crater': {'In Situ Science': 4,'Distance to Water ISRU [km]': 450, 'Distance to Other Geological Regions [km]': 130},
#     'Nili Fossae': {'In Situ Science': 4,'Distance to Water ISRU [km]': 1000},
    'Northeast Syrtis': {'In Situ Science': 3.9,'Distance to Water ISRU [km]': 450,'Distance to Other Geological Regions [km]': 200},
#     'Eberswalde Crater': {'In Situ Science': 4,'Distance to Water ISRU [km]': 1000},
    'Columbia Hills': {'In Situ Science': 2.8,'Distance to Water ISRU [km]': 138,'Distance to Other Geological Regions [km]': 50},
    'Midway': {'In Situ Science': 2.8,'Distance to Water ISRU [km]': 450,'Distance to Other Geological Regions [km]': 150}
}

df_2['In Situ Science Score'] = df_2['Landing Location'].apply(lambda x: landing_dependencies[x]['In Situ Science'])
df_2['Distance from Water ISRU [km]'] = df_2['Landing Location'].apply(lambda x: landing_dependencies[x]['Distance to Water ISRU [km]'])
df_2['Distance from Other Geological [km]'] = df_2['Landing Location'].apply(lambda x: landing_dependencies[x]['Distance to Other Geological Regions [km]'])

reference_df['In Situ Science Score'] = reference_df['Landing Location'].apply(lambda x: landing_dependencies[x]['In Situ Science'])
reference_df['Distance from Water ISRU [km]'] = reference_df['Landing Location'].apply(lambda x: landing_dependencies[x]['Distance to Water ISRU [km]'])
reference_df['Distance from Other Geological [km]'] = reference_df['Landing Location'].apply(lambda x: landing_dependencies[x]['Distance to Other Geological Regions [km]'])



## EDL Dependencies: Payload, Cost, Mass, Accuracy, Reusable, Risk

In [58]:
edl_depend = {
    "Retropropulsion": {'Payload': 265, 'Cost': 100, 'Mass': 280, 'Accuracy': 100,'Reusable': "Yes", 'Risk': 2},
    "Supersonic Decelerators": {'Payload': 84, 'Cost': 20, 'Mass': 86, 'Accuracy': 5000,'Reusable': "No", 'Risk': 3},
    "Hypersonic Aeroassist": {'Payload': 40.4, 'Cost': 100, 'Mass': 110.2, 'Accuracy': 100,'Reusable': "Yes", 'Risk': 1},
    "Deployable Entry System": {'Payload': 107, 'Cost': 50, 'Mass': 115, 'Accuracy': 1000, 'Reusable': "No", 'Risk': 4},
}

df_2['EDL Payload [mT]'] = df_2['EDL'].apply(lambda x: edl_depend[x]['Payload'])
df_2['EDL Cost [$M]'] = df_2['EDL'].apply(lambda x: edl_depend[x]['Cost'])
df_2['EDL System Mass [mT]'] = df_2['EDL'].apply(lambda x: edl_depend[x]['Mass'])
df_2['EDL System Accuracy [m]'] = df_2['EDL'].apply(lambda x: edl_depend[x]['Accuracy'])
df_2['EDL Reusable'] = df_2['EDL'].apply(lambda x: edl_depend[x]['Reusable'])
df_2['EDL Risk'] = df_2['EDL'].apply(lambda x: edl_depend[x]['Risk'])

reference_df['EDL Payload [mT]'] = reference_df['EDL'].apply(lambda x: edl_depend[x]['Payload'])
reference_df['EDL Cost [$M]'] = reference_df['EDL'].apply(lambda x: edl_depend[x]['Cost'])
reference_df['EDL System Mass [mT]'] = reference_df['EDL'].apply(lambda x: edl_depend[x]['Mass'])
reference_df['EDL System Accuracy [m]'] = reference_df['EDL'].apply(lambda x: edl_depend[x]['Accuracy'])
reference_df['EDL Reusable'] = reference_df['EDL'].apply(lambda x: edl_depend[x]['Reusable'])
reference_df['EDL Risk'] = reference_df['EDL'].apply(lambda x: edl_depend[x]['Risk'])

# Just for Clarity Sake, lets take a look at all of the variables we have to work with (Beyond the variables we traded on above - i.e. just the dependent variables)

## Total Crew Surace Days
## Consumables Needed [kg]
## Minimum Number of Rovers Needed
## Rover Mass [kg]
## Rover People Capacity
## Rover Payload Capacity [mT]
## Rover Exploration Range [km]
## Rover Pressurized
## Rover Cost [Billions]
## Landing Science Score
## Distance From Water ISRU [km]
## EDL Payload [mT]
## EDL Cost [Millions]
## EDL System Mass [mT]
## EDL System Accuracy [m]
## EDL Reusable
## EDL Risk

In [59]:
df_2.head()

,EDL,Landing Location,Number of Crew,Rover,Total Crew Days,Consumables Needed [kg],Habitation Volume [m^3],Number of Rovers Needed,Rover Mass [kg],Rover People Capacity,...,Rover Cost [$B],In Situ Science Score,Distance from Water ISRU [km],Distance from Other Geological [km],EDL Payload [mT],EDL Cost [$M],EDL System Mass [mT],EDL System Accuracy [m],EDL Reusable,EDL Risk
0,Retropropulsion,Jezero Crater,2,MSC,1000,41837.50,50,2,185,1,...,16,4.0,450,130,265.0,100,280.0,100,Yes,2
1,Retropropulsion,Jezero Crater,3,MSC,1500,62756.25,75,2,185,1,...,16,4.0,450,130,265.0,100,280.0,100,Yes,2
2,Retropropulsion,Jezero Crater,4,MSC,2000,83675.00,100,2,185,1,...,16,4.0,450,130,265.0,100,280.0,100,Yes,2
3,Retropropulsion,Jezero Crater,5,MSC,2500,104593.75,125,2,185,1,...,16,4.0,450,130,265.0,100,280.0,100,Yes,2
4,Retropropulsion,Jezero Crater,6,MSC,3000,125512.50,150,2,185,1,...,16,4.0,450,130,265.0,100,280.0,100,Yes,2


In [60]:
reference_df

,EDL,Landing Location,Number of Crew,Rover,Total Crew Days,Consumables Needed [kg],Habitation Volume [m^3],Number of Rovers Needed,Rover Mass [kg],Rover People Capacity,...,EDL Cost [$M],EDL System Mass [mT],EDL System Accuracy [m],EDL Reusable,EDL Risk,Mass on Surface [kg],Number of Launches,Basic Cost [$B],Survivability Metric,Exploration Score
NASA DRM 5.0,Hypersonic Aeroassist,Jezero Crater,6,Commuter,3000,125512.5,197.73,2,8000,2,...,100,110.2,100,Yes,1,146487.5,4,93.946,0.000741,134656.17797


# Given the variables we have to work with, lets build out some useful metrics

# Metrics:

## Mass on Surface: Consumables, rover mass, edl mass, hab mass
## Basic Cost: Rover Cost, Hab Cost, EDL Cost
## Survivability
## Exploration Score


In [61]:
df_3 = df_2.copy() #Create a copy for new formulaic variables

In [62]:
def mass_on_surface_excluding_edl(df):
    consumables = df['Consumables Needed [kg]']
    rover_mass = df['Rover Mass [kg]']*df['Number of Rovers Needed']
    hab_mass = df['Habitation Volume [m^3]']*65 #If same ratio as hab volume to mass as DRM 5.0
    l = [consumables,rover_mass,hab_mass]
#     other_mass = 
    return sum(l)

df_3['Mass on Surface [kg]'] = df_3.apply(mass_on_surface_excluding_edl,axis = 1)
reference_df['Mass on Surface [kg]'] = reference_df['Consumables Needed [kg]'] + reference_df['Rover Mass [kg]'] + 12975

In [63]:
reference_df

,EDL,Landing Location,Number of Crew,Rover,Total Crew Days,Consumables Needed [kg],Habitation Volume [m^3],Number of Rovers Needed,Rover Mass [kg],Rover People Capacity,...,EDL Cost [$M],EDL System Mass [mT],EDL System Accuracy [m],EDL Reusable,EDL Risk,Mass on Surface [kg],Number of Launches,Basic Cost [$B],Survivability Metric,Exploration Score
NASA DRM 5.0,Hypersonic Aeroassist,Jezero Crater,6,Commuter,3000,125512.5,197.73,2,8000,2,...,100,110.2,100,Yes,1,146487.5,4,93.946,0.000741,134656.17797


## Determine how many launches based on EDL payload constraint

In [64]:
df_4 = df_3.copy()
df_4['Number of Launches'] = df_4.apply(lambda x: math.ceil(x['Mass on Surface [kg]']/(x['EDL Payload [mT]']*1000)), axis = 1)

reference_df['Number of Launches'] = reference_df.apply(lambda x: math.ceil(x['Mass on Surface [kg]']/(x['EDL Payload [mT]']*1000)), axis = 1)



In [65]:
reference_df.head()

,EDL,Landing Location,Number of Crew,Rover,Total Crew Days,Consumables Needed [kg],Habitation Volume [m^3],Number of Rovers Needed,Rover Mass [kg],Rover People Capacity,...,EDL Cost [$M],EDL System Mass [mT],EDL System Accuracy [m],EDL Reusable,EDL Risk,Mass on Surface [kg],Number of Launches,Basic Cost [$B],Survivability Metric,Exploration Score
NASA DRM 5.0,Hypersonic Aeroassist,Jezero Crater,6,Commuter,3000,125512.5,197.73,2,8000,2,...,100,110.2,100,Yes,1,146487.5,4,93.946,0.000741,134656.17797


In [66]:
def basic_cost_metric(df):
    edl_cost = df['Number of Launches']*df['EDL Cost [$M]']*0.001 #conversion to billions
    rover_cost = df['Number of Rovers Needed']*df['Rover Cost [$B]']
    hab_cost = df['Habitation Volume [m^3]']*0.2 #Assume 1 m^3 cost 200 Million
    return edl_cost + rover_cost + hab_cost
df_4['Basic Cost [$B]'] = df_4.apply(basic_cost_metric,axis = 1)
reference_df['Basic Cost [$B]'] = reference_df.apply(basic_cost_metric,axis = 1)

In [67]:
df_4.columns

Index(['EDL', 'Landing Location', 'Number of Crew', 'Rover', 'Total Crew Days',
       'Consumables Needed [kg]', 'Habitation Volume [m^3]',
       'Number of Rovers Needed', 'Rover Mass [kg]', 'Rover People Capacity',
       'Rover Payload Capacity [mT]', 'Rover Range (km)', 'Rover Pressurized',
       'Rover Cost [$B]', 'In Situ Science Score',
       'Distance from Water ISRU [km]', 'Distance from Other Geological [km]',
       'EDL Payload [mT]', 'EDL Cost [$M]', 'EDL System Mass [mT]',
       'EDL System Accuracy [m]', 'EDL Reusable', 'EDL Risk',
       'Mass on Surface [kg]', 'Number of Launches', 'Basic Cost [$B]'],
      dtype='object')

In [68]:
def survivability_score(df):
    isru_close = df['Distance from Water ISRU [km]']
    edl_risk = df['EDL Risk']
    
    #units: 1/km*% so 1/km
    #Radiation protection - see how landing location can impact this
    
    return 1/((isru_close)*edl_risk)

df_4['Survivability Metric'] = df_4.apply(survivability_score,axis = 1)
df_4['Normalized Survivability Metric [%]'] = 100*df_4['Survivability Metric']/(df_4["Survivability Metric"].max()) #Normalized

reference_df['Survivability Metric'] = reference_df.apply(survivability_score,axis = 1)


In [69]:
df_4.head()

,EDL,Landing Location,Number of Crew,Rover,Total Crew Days,Consumables Needed [kg],Habitation Volume [m^3],Number of Rovers Needed,Rover Mass [kg],Rover People Capacity,...,EDL Cost [$M],EDL System Mass [mT],EDL System Accuracy [m],EDL Reusable,EDL Risk,Mass on Surface [kg],Number of Launches,Basic Cost [$B],Survivability Metric,Normalized Survivability Metric [%]
0,Retropropulsion,Jezero Crater,2,MSC,1000,41837.50,50,2,185,1,...,100,280.0,100,Yes,2,45457.50,1,42.1,0.001111,15.333333
1,Retropropulsion,Jezero Crater,3,MSC,1500,62756.25,75,2,185,1,...,100,280.0,100,Yes,2,68001.25,1,47.1,0.001111,15.333333
2,Retropropulsion,Jezero Crater,4,MSC,2000,83675.00,100,2,185,1,...,100,280.0,100,Yes,2,90545.00,1,52.1,0.001111,15.333333
3,Retropropulsion,Jezero Crater,5,MSC,2500,104593.75,125,2,185,1,...,100,280.0,100,Yes,2,113088.75,1,57.1,0.001111,15.333333
4,Retropropulsion,Jezero Crater,6,MSC,3000,125512.50,150,2,185,1,...,100,280.0,100,Yes,2,135632.50,1,62.1,0.001111,15.333333


In [70]:
def exploration_score(df):
    landing_science_score = df['In Situ Science Score']
    landing_geo_score = df['Distance from Other Geological [km]']
    rover_range = df['Rover Range (km)']
    num_crew = df['Number of Crew']
    edl_accuracy = df['EDL System Accuracy [m]']
    
    #Logic: Want high landing score, high rover range, high edl accuracy (low number)
    #The number of crew lets apply that sqrt and see what happens
    #Units = km/km^2
    
    return (landing_science_score*rover_range)*((num_crew*500)**0.5)/((edl_accuracy*0.001)*landing_geo_score**0.1)

df_4['Exploration Score'] = df_4.apply(exploration_score,axis = 1)
df_4['Normalized Exploration Score [%]'] = 100*df_4['Exploration Score']/max(df_4['Exploration Score'])

reference_df['Exploration Score'] = reference_df.apply(exploration_score, axis = 1)

In [71]:
reference_df

,EDL,Landing Location,Number of Crew,Rover,Total Crew Days,Consumables Needed [kg],Habitation Volume [m^3],Number of Rovers Needed,Rover Mass [kg],Rover People Capacity,...,EDL Cost [$M],EDL System Mass [mT],EDL System Accuracy [m],EDL Reusable,EDL Risk,Mass on Surface [kg],Number of Launches,Basic Cost [$B],Survivability Metric,Exploration Score
NASA DRM 5.0,Hypersonic Aeroassist,Jezero Crater,6,Commuter,3000,125512.5,197.73,2,8000,2,...,100,110.2,100,Yes,1,146487.5,4,93.946,0.002222,134656.17797


## Lets normalize these metrics based of the drm 5 mission

In [74]:
df_4['Normalized Survivability with DRM 5'] = df_4['Survivability Metric'].apply(lambda x: x/(reference_df['Survivability Metric'][0]))
df_4['Normalized Exploration with DRM 5'] = df_4['Exploration Score'].apply(lambda x: x/(reference_df['Exploration Score'][0]))




In [75]:
df_4.head()

,EDL,Landing Location,Number of Crew,Rover,Total Crew Days,Consumables Needed [kg],Habitation Volume [m^3],Number of Rovers Needed,Rover Mass [kg],Rover People Capacity,...,EDL Risk,Mass on Surface [kg],Number of Launches,Basic Cost [$B],Survivability Metric,Normalized Survivability Metric [%],Exploration Score,Normalized Exploration Score [%],Normalized Survivability with DRM 5,Normalized Exploration with DRM 5
0,Retropropulsion,Jezero Crater,2,MSC,1000,41837.50,50,2,185,1,...,2,45457.50,1,42.1,0.001111,15.333333,124390.048959,12.316806,0.5,0.923760
1,Retropropulsion,Jezero Crater,3,MSC,1500,62756.25,75,2,185,1,...,2,68001.25,1,47.1,0.001111,15.333333,152346.074514,15.084945,0.5,1.131371
2,Retropropulsion,Jezero Crater,4,MSC,2000,83675.00,100,2,185,1,...,2,90545.00,1,52.1,0.001111,15.333333,175914.094261,17.418594,0.5,1.306395
3,Retropropulsion,Jezero Crater,5,MSC,2500,104593.75,125,2,185,1,...,2,113088.75,1,57.1,0.001111,15.333333,196677.936484,19.474580,0.5,1.460593
4,Retropropulsion,Jezero Crater,6,MSC,3000,125512.50,150,2,185,1,...,2,135632.50,1,62.1,0.001111,15.333333,215449.884752,21.333333,0.5,1.600000


## Try this we multiply Survivability by Exploration Score, lets see what happens

In [43]:
df_4['Test Metric'] = df_4['Normalized Exploration Score [%]'] * df_4['Normalized Survivability Metric [%]']

## Plots

### Basic Cost vs Normalized Exploration Score
### Basic Cost vs Normalized Survivability
### Normalized Survivability vs Exploration Score

In [77]:
data = []
colors = ['red','blue','orange','green','black']
landing_locs = list(set(df_4['Landing Location'].values))
for i in range(len(landing_locs)):
    new_df = df_4[df_4['Landing Location'] == landing_locs[i]]
    
    text_list = []
    for index, row in new_df.iterrows():
        num_crew = row['Number of Crew']
        surv_score = row['Normalized Survivability Metric [%]']
        edl = row['EDL']
        rover = row['Rover']

        
        text = ('Number of Crew ' + str(num_crew) + '<br>'
                + 'Norm Survivability Score ' + str(surv_score) +'<br>'
                + 'EDL ' + edl +'<br>'
                + 'Arch Index ' + str(index)
                + 'Rover Selection '+ rover)
        text_list.append(text)

    new_df['Text'] = text_list
    
    new_data = go.Scatter(x=list(new_df['Basic Cost [$B]'].values),
                          y=list(new_df['Normalized Exploration Score [%]']),
                          mode = 'markers',
                          marker=dict(
                              size = 9,
                              color=colors[i],
                              colorscale='Viridis',
                              showscale=False),
                          text = new_df['Text'],
                          name = str(landing_locs[i])
                         )
    data.append(new_data)
# data
updatemenus = list([
    dict(active=-1,
         buttons=list([   
            dict(label = 'Columbia Hills',
                 method = 'update',
                 args = [{'visible': [False, False, True, False]},
                         {'title': 'Columbia Hills',
                          'annotations': []}]),
            dict(label = 'Midway',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]},
                         {'title': 'Eberswalde Crater',
                          'annotations': []}]),
            dict(label = 'Jezero Crater',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]},
                         {'title': 'Jezero Crater',
                          'annotations': []}]),
#             dict(label = 'Nili Fossae',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, True, False]},
#                          {'title': 'Nili Fossae',
#                           'annotations': []}]),
             dict(label = 'Northeast Syrtis',
                  method = 'update',
                  args = [{'visible': [True, False, False, False]},
                          {'title': 'Northeast Syrtis',
                           'annotations': []}]),
             dict(label = 'All Landing Locations',
                  method = 'update',
                  args = [{'visible': [True, True, True, True]},
                          {'title': 'All Landing Locations',
                           'annotations': []}])
        ]),
    )
])

layout = dict(title='Exploration Score vs Basic Cost [$B]', showlegend=True,
              updatemenus=updatemenus,xaxis = {'title': 'Basic Cost in Billions'},
             yaxis = {'title': 'Exploration Score'})
fig = dict(data=data, layout=layout)
iplot(fig)


/anaconda3/envs/eric_env/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [78]:
df_4.loc[384]

EDL                                    Deployable Entry System
Landing Location                                 Jezero Crater
Number of Crew                                               6
Rover                                                 Commuter
Total Crew Days                                           3000
Consumables Needed [kg]                                 125512
Habitation Volume [m^3]                                    150
Number of Rovers Needed                                      2
Rover Mass [kg]                                           8000
Rover People Capacity                                        2
Rover Payload Capacity [mT]                               22.6
Rover Range (km)                                           100
Rover Pressurized                                          Yes
Rover Cost [$B]                                             27
In Situ Science Score                                        4
Distance from Water ISRU [km]                          

In [51]:
data = []
colors = ['red','blue','orange','green','black']
edl_options = list(set(df_4['EDL'].values))
for i in range(len(landing_locs)):
    new_df = df_4[df_4['EDL'] == edl_options[i]]
    
    text_list = []
    for index, row in new_df.iterrows():
        num_crew = row['Number of Crew']
        exp_score = row['Normalized Exploration Score [%]']
        landing_loc = row['Landing Location']
        
        text = ('Number of Crew ' + str(num_crew) + '<br>'
                + 'Norm Exploration Score ' + str(exp_score) +'<br>'
                + 'Landing Location ' + landing_loc+'<br>'
                + "Index " + str(index))
        text_list.append(text)

    new_df['Text'] = text_list
    
    new_data = go.Scatter(x=list(new_df['Basic Cost [$B]'].values),
                          y=list(new_df['Normalized Survivability Metric [%]']),
                          mode = 'markers',
                          marker=dict(
                              size = 9,
                              color=colors[i],
                              colorscale='Viridis',
                              showscale=False),
                          text = new_df['Text'],
                          name = str(edl_options[i])
                         )
    data.append(new_data)
# data
updatemenus = list([
    dict(active=-1,
         buttons=list([   
            dict(label = 'Hypersonic Aeroassist',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]},
                         {'title': 'Hypersonic Aeroassist',
                          'annotations': []}]),
            dict(label = 'Supersonic Decelerators',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]},
                         {'title': 'Supersonic Decelerators',
                          'annotations': []}]),
            dict(label = 'Retropropulsion',
                 method = 'update',
                 args = [{'visible': [True, False, False, False]},
                         {'title': 'Retropropulsion',
                          'annotations': []}]),
             dict(label = 'Deployable Entry',
                  method = 'update',
                  args = [{'visible': [False, False, True, False]},
                          {'title': 'Deployable Entry',
                           'annotations': []}]),
             dict(label = 'All EDL Options',
                  method = 'update',
                  args = [{'visible': [True, True, True, True]},
                          {'title': 'All EDL Options',
                           'annotations': []}])
        ]),
    )
])

layout = dict(title='Normalized Survivability vs Basic Cost [$B]', showlegend=True,
              updatemenus=updatemenus,xaxis = {'title': 'Basic Cost in Billions'},
             yaxis = {'title': 'Normalized Survivability'})
fig = dict(data=data, layout=layout)
iplot(fig)

/anaconda3/envs/eric_env/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [53]:
data = []
colors = ['red','blue','orange','green','black']
landing_locations = list(set(df_4['Landing Location'].values))
for i in range(len(landing_locs)):
    new_df = df_4[df_4['Landing Location'] == landing_locations[i]]
    text_list = []
    for index, row in new_df.iterrows():
        num_crew = row['Number of Crew']
        basic_cost = row['Basic Cost [$B]']
        edl = row['EDL']
        
        text = ('Number of Crew ' + str(num_crew) + '<br>' + 'Basic Cost ' + str(basic_cost) +'<br>'
                + 'EDL ' + edl+'<br>' +'Index ' +str(index))
        text_list.append(text)

    new_df['Text'] = text_list
    
    new_data = go.Scatter(x=list(new_df['Normalized Survivability Metric [%]'].values),
                          y=list(new_df['Normalized Exploration Score [%]']),
                          mode = 'markers',
                          marker=dict(
                              size = 9,
                              color=colors[i],
                              colorscale='Viridis',
                              showscale=False),
                          text = new_df['Text'],
                          name = str(landing_locations[i])
                         )
    data.append(new_data)

# data
updatemenus = list([
    dict(active=-1,
         buttons=list([   
            dict(label = 'Columbia Hills',
                 method = 'update',
                 args = [{'visible': [False, False, True, False]},
                         {'title': 'Columbia Hills',
                          'annotations': []}]),
            dict(label = 'Midway',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]},
                         {'title': 'Eberswalde Crater',
                          'annotations': []}]),
            dict(label = 'Jezero Crater',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]},
                         {'title': 'Jezero Crater',
                          'annotations': []}]),
#             dict(label = 'Nili Fossae',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, True, False]},
#                          {'title': 'Nili Fossae',
#                           'annotations': []}]),
             dict(label = 'Northeast Syrtis',
                  method = 'update',
                  args = [{'visible': [True, False, False, False]},
                          {'title': 'Northeast Syrtis',
                           'annotations': []}]),
             dict(label = 'All Landing Locations',
                  method = 'update',
                  args = [{'visible': [True, True, True, True]},
                          {'title': 'All Landing Locations',
                           'annotations': []}])
        ]),
    )
])

layout = dict(title='Normalized Survivability vs Normalized Exploration', showlegend=True,
              updatemenus=updatemenus,xaxis = {'title': 'Normalized Survivability'},
             yaxis = {'title': 'Normalized Exploration'})
fig = dict(data=data, layout=layout)
iplot(fig)

/anaconda3/envs/eric_env/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [56]:
data = []
colors = ['red','blue','orange','green','black']
landing_locs = list(set(df_4['Landing Location'].values))
for i in range(len(landing_locs)):
    new_df = df_4[df_4['Landing Location'] == landing_locs[i]]
    
#     new_df['Text'] = ['Number of Crew ' + str(n) for n in list(new_df['Number of Crew'].values)]
    
    text_list = []
    for index, row in new_df.iterrows():
        num_crew = row['Number of Crew']
        norm_surv = row['Normalized Survivability Metric [%]']
        edl = row['EDL']
        
        text = ('Number of Crew ' + str(num_crew) + '<br>' + 'Normalized Survivability ' + str(norm_surv) +'<br>'
                + 'EDL ' + edl+'<br>'
                + 'Index ' + str(index))
        text_list.append(text)

    new_df['Text'] = text_list
        
    new_data = go.Scatter(x=list(new_df['Mass on Surface [kg]'].values),
                          y=list(new_df['Normalized Exploration Score [%]']),
                          mode = 'markers',
                          marker=dict(
                              size = 9,
                              color=colors[i],
                              colorscale='Viridis',
                              showscale=False),
                          text = new_df['Text'],
                          name = str(landing_locs[i])
                         )
    data.append(new_data)
# data
updatemenus = list([
    dict(active=-1,
         buttons=list([   
            dict(label = 'Columbia Hills',
                 method = 'update',
                 args = [{'visible': [False, False, True, False]},
                         {'title': 'Columbia Hills',
                          'annotations': []}]),
            dict(label = 'Midway',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]},
                         {'title': 'Eberswalde Crater',
                          'annotations': []}]),
            dict(label = 'Jezero Crater',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]},
                         {'title': 'Jezero Crater',
                          'annotations': []}]),
#             dict(label = 'Nili Fossae',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, True, False]},
#                          {'title': 'Nili Fossae',
#                           'annotations': []}]),
             dict(label = 'Northeast Syrtis',
                  method = 'update',
                  args = [{'visible': [True, False, False, False]},
                          {'title': 'Northeast Syrtis',
                           'annotations': []}]),
             dict(label = 'All Landing Locations',
                  method = 'update',
                  args = [{'visible': [True, True, True, True]},
                          {'title': 'All Landing Locations',
                           'annotations': []}])
        ]),
    )
])

layout = dict(title='Exploration Score vs Mass on Surface', showlegend=True,
              updatemenus=updatemenus,xaxis = {'title': 'Mass on Surface'},
             yaxis = {'title': 'Exploration Score'})
fig = dict(data=data, layout=layout)
iplot(fig)

/anaconda3/envs/eric_env/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [37]:
df_4.loc[[120,130]]

,EDL,Landing Location,Number of Crew,Rover,Total Crew Days,Consumables Needed [kg],Habitation Volume [m^3],Number of Rovers Needed,Rover Mass [kg],Rover People Capacity,...,Mass on Surface [kg],Number of Launches,Basic Cost [$B],Survivability Metric,Normalized Survivability Metric [%],Exploration Score,Normalized Exploration Score [%],Normalized Survivability with DRM 5,Normalized Exploration with DRM 5,Test Metric
120,Hypersonic Aeroassist,Jezero Crater,2,Mars Direct,1000,41837.5,50,1,1400,4,...,46487.5,2,54.2,0.000741,10.222222,26076.056737,57.735027,1.0,4.330127,590.180275
130,Hypersonic Aeroassist,Northeast Syrtis,2,Mars Direct,1000,41837.5,50,1,1400,4,...,46487.5,2,54.2,0.000741,10.222222,24352.181317,53.918192,1.0,4.043864,551.163744
